# Automated Vulnerability Scanner for Security Operations

This Jupyter Notebook is the development environment for my CYB 333 course project. It is used to automate the process of identifying open ports on network hosts, collect vulnerability data, and analyze results using Python-based tools. This notebook demonstrates how Jupyter supports cybersecurity automation through code execution, documentation, and visualization.

In [1]:
# Environment verification and library import
import sys
import platform
import requests
import json

print("Python version:", sys.version)
print("Platform:", platform.system(), platform.release())
print("Requests library version:", requests.__version__)
print("Environment successfully initialized for security automation.")

Python version: 3.10.19 | packaged by conda-forge | (main, Oct 13 2025, 14:05:01) [MSC v.1944 64 bit (AMD64)]
Platform: Windows 10
Requests library version: 2.32.5
Environment successfully initialized for security automation.


## Project Workflow Overview

This notebook follows an iterative security automation workflow aligned with Week 3 course materials on Jupyter Notebooks:

1. **Data Collection:** Retrieve information from network hosts using Python-based scanning techniques.
2. **Preprocessing:** Structure and normalize the scanned results for analysis.
3. **Analysis & Visualization:** Identify potentially vulnerable services and visualize findings using charts.
4. **Reporting & Documentation:** Export results and document findings for security operations and remediation actions.

This demonstrates how Jupyter Notebooks support real-time analysis, iterative testing, and transparency in cybersecurity workflows.

## Development Milestone – October 19, 2025

Today marks the initialization of the development phase for the Automated Vulnerability Scanner project. The environment has been successfully configured, project workflow has been defined, and the notebook has been prepared to begin implementing security automation functions in the next phase.

In [2]:
# Placeholder function to simulate network scanning
def scan_host(ip_address):
    # In the full implementation, this function will perform real Nmap or socket-based scans.
    # For now, it simulates an open port response for demonstration purposes.
    print(f"Scanning host: {ip_address}")
    simulated_results = {
        "host": ip_address,
        "status": "up",
        "open_ports": [22, 80, 443],
        "message": "Simulated scan complete. Real scan functionality to be implemented next."
    }
    return simulated_results

# Test the placeholder function
test_result = scan_host("192.168.1.10")
print("Scan Results:", test_result)

Scanning host: 192.168.1.10
Scan Results: {'host': '192.168.1.10', 'status': 'up', 'open_ports': [22, 80, 443], 'message': 'Simulated scan complete. Real scan functionality to be implemented next.'}


## Milestone: Initial Scanning Function Implemented

The first development function has been implemented to simulate scanning a host for open ports. While this is a placeholder demonstration, it establishes the core logic structure that will be expanded to perform real network scans using libraries such as `python-nmap` and `socket`. This marks the official beginning of the development phase for the automated vulnerability scanner.

In [2]:
# Real safe localhost scan using python-nmap (with fallback)
import json
from pprint import pprint

def real_scan_localhost():
    try:
        import nmap  # python-nmap
    except Exception as e:
        print("python-nmap not available in this kernel:", e)
        return None

    scanner = nmap.PortScanner()
    target = "127.0.0.1"
    print(f"Running safe nmap scan against {target} (localhost)...")
    try:
        # -sT uses TCP connect scan (works without raw sockets privileges)
        scanner.scan(hosts=target, arguments='-sT -Pn -T4')
        if target in scanner.all_hosts():
            result = scanner[target]
            return result
        else:
            print("No host information found in scanner result.")
            return None
    except Exception as exc:
        print("Nmap scan failed:", exc)
        return None

scan_output = real_scan_localhost()
if scan_output:
    print("Real scan output (sample):")
    pprint(scan_output)
else:
    print("Real scan not available. Showing simulated scan result instead.")
    simulated_results = {
        "host": "127.0.0.1",
        "status": "up",
        "open_ports": [22, 80, 443],
        "message": "Simulated scan (fallback): install nmap and retry for live results."
    }
    pprint(simulated_results)

Running safe nmap scan against 127.0.0.1 (localhost)...
Real scan output (sample):
{'addresses': {'ipv4': '127.0.0.1'},
 'hostnames': [{'name': 'activate.adobe.com', 'type': 'PTR'}],
 'status': {'reason': 'user-set', 'state': 'up'},
 'tcp': {135: {'conf': '3',
               'cpe': '',
               'extrainfo': '',
               'name': 'msrpc',
               'product': '',
               'reason': 'syn-ack',
               'state': 'open',
               'version': ''},
         445: {'conf': '3',
               'cpe': '',
               'extrainfo': '',
               'name': 'microsoft-ds',
               'product': '',
               'reason': 'syn-ack',
               'state': 'open',
               'version': ''},
         2179: {'conf': '3',
                'cpe': '',
                'extrainfo': '',
                'name': 'vmrdp',
                'product': '',
                'reason': 'syn-ack',
                'state': 'open',
                'version': ''},
         53

In [1]:
# Port range scanning cell (localhost) - Automated Vulnerability Scanner
from pprint import pprint
import json, os
from datetime import datetime

REPORTS_DIR = "../reports"
os.makedirs(REPORTS_DIR, exist_ok=True)

def port_range_scan(target="127.0.0.1", port_range="1-1024"):
    try:
        import nmap
    except Exception as e:
        print("python-nmap not available in this kernel:", e)
        return {"error": "python-nmap not available", "simulated": True}

    scanner = nmap.PortScanner()
    try:
        print(f"Running nmap scan against {target} on ports {port_range} ...")
        scanner.scan(hosts=target, arguments=f"-sT -Pn -p {port_range} -T4")
        if target not in scanner.all_hosts():
            print("No host info returned by nmap for target.")
            return {"error": "no_host_info", "simulated": True}

        host_result = scanner[target]
        open_ports = []
        if 'tcp' in host_result:
            for port, info in host_result['tcp'].items():
                if info.get('state') == 'open':
                    open_ports.append({
                        "port": int(port),
                        "service": info.get('name'),
                        "reason": info.get('reason'),
                        "product": info.get('product'),
                    })

        result = {
            "target": target,
            "status": host_result.get('status'),
            "open_ports": sorted(open_ports, key=lambda x: x['port']),
            "raw": host_result
        }
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = os.path.join(REPORTS_DIR, f"scan_{target.replace('.', '_')}_{timestamp}.json")
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(result, f, indent=2)
        print(f"Scan complete — results saved to: {filename}")
        return result

    except Exception as exc:
        print("Nmap scan raised an exception:", exc)
        return {"error": str(exc), "simulated": True}

# run it
scan_result = port_range_scan(target="127.0.0.1", port_range="1-1024")
pprint(scan_result)

Running nmap scan against 127.0.0.1 on ports 1-1024 ...
Scan complete — results saved to: ../reports\scan_127_0_0_1_20251024_231920.json
{'open_ports': [{'port': 135,
                 'product': '',
                 'reason': 'syn-ack',
                 'service': 'msrpc'},
                {'port': 445,
                 'product': '',
                 'reason': 'syn-ack',
                 'service': 'microsoft-ds'},
                {'port': 623,
                 'product': '',
                 'reason': 'syn-ack',
                 'service': 'oob-ws-http'}],
 'raw': {'addresses': {'ipv4': '127.0.0.1'},
         'hostnames': [{'name': 'activate.adobe.com', 'type': 'PTR'}],
         'status': {'reason': 'user-set', 'state': 'up'},
         'tcp': {135: {'conf': '3',
                       'cpe': '',
                       'extrainfo': '',
                       'name': 'msrpc',
                       'product': '',
                       'reason': 'syn-ack',
                       'state

## Milestone: Port Range Scanning Implemented (Oct 24, 2025)

Implemented a port-range scanning function that performs a TCP connect scan over ports 1–1024 using `python-nmap`. Results are parsed, summarized, and saved to the `reports/` folder as JSON. This moves the scanner from a single-host proof-of-concept to a functional scanner capable of enumerating open services across a port range. All scans were conducted only against localhost (127.0.0.1) to comply with ethical guidelines.

In [2]:
# CVE lookup (local mapping) - Annotate last scan results with sample CVEs
import glob, json, os
from pprint import pprint

# small sample mapping: service or port -> example CVEs (illustrative)
CVE_MAP = {
    "ssh": [{"cve": "CVE-2016-0777", "desc": "OpenSSH potential info leak (example)"}],
    "http": [{"cve": "CVE-2020-12345", "desc": "Example HTTP vuln"}],
    "microsoft-ds": [{"cve": "CVE-2020-0796", "desc": "SMB vulnerability example"}],
    445: [{"cve": "CVE-2020-0796", "desc": "SMB vulnerability example"}],
    135: [{"cve": "CVE-2003-0352", "desc": "MSRPC example CVE"}],
    623: [{"cve": "CVE-2019-0708", "desc": "Example OOB management interface vuln"}],
    8888: [{"cve": "CVE-2019-0001", "desc": "Example service vuln"}],
    16992: [{"cve": "CVE-2017-0144", "desc": "Remote code execution example"}],
    # add or customize pairs as you like
}

# find the latest report for localhost
reports = sorted(glob.glob("../reports/scan_127_0_0_1_*.json"))
if not reports:
    print("No reports found in ../reports. Run the port-range scan first.")
else:
    latest = reports[-1]
    print("Loading report:", latest)
    with open(latest, "r", encoding="utf-8") as f:
        data = json.load(f)

    annotated = []
    for port_info in data.get("open_ports", []):
        port = port_info.get("port")
        service = port_info.get("service")
        matches = []
        # check by port number
        if port in CVE_MAP:
            matches.extend(CVE_MAP[port])
        # check by service name
        if service and service in CVE_MAP:
            matches.extend(CVE_MAP[service])
        annotated.append({
            "port": port,
            "service": service,
            "matches": matches
        })

    result = {
        "target": data.get("target"),
        "status": data.get("status"),
        "annotated_open_ports": annotated
    }

    pprint(result)
    # save annotated report
    annotated_file = latest.replace(".json", "_annotated.json")
    with open(annotated_file, "w", encoding="utf-8") as f:
        json.dump(result, f, indent=2)
    print("Annotated report saved to:", annotated_file)

Loading report: ../reports\scan_127_0_0_1_20251024_231920.json
{'annotated_open_ports': [{'matches': [{'cve': 'CVE-2003-0352',
                                        'desc': 'MSRPC example CVE'}],
                           'port': 135,
                           'service': 'msrpc'},
                          {'matches': [{'cve': 'CVE-2020-0796',
                                        'desc': 'SMB vulnerability example'},
                                       {'cve': 'CVE-2020-0796',
                                        'desc': 'SMB vulnerability example'}],
                           'port': 445,
                           'service': 'microsoft-ds'},
                          {'matches': [{'cve': 'CVE-2019-0708',
                                        'desc': 'Example OOB management '
                                                'interface vuln'}],
                           'port': 623,
                           'service': 'oob-ws-http'}],
 'status': {'reason': 'user-set',